In [1]:
import csv
import os
from datetime import datetime


class Pronostiek:
    def __init__(self):
        self.juist = self._lees_csv('Juist.csv')
        self.personen = {}
        self.bestanden = {}
        self.betaald = {}
        self._laad_spelers('spelers')

    def _lees_csv(self, pad):
        with open(pad, 'r') as infile:
            return [rij for rij in csv.reader(infile, delimiter=';')]

    def _laad_spelers(self, spelers_dir):
        for filename in os.listdir(spelers_dir):
            if filename.endswith('.csv'):
                voornaam, naam = filename.split('.')[:2]
                self.nieuwe_speler(voornaam, naam, spelers_dir=spelers_dir)

    def nieuwe_speler(self, voornaam, naam, betaald=True, spelers_dir='spelers'):
        volledige_naam = f'{voornaam} {naam}'
        self.personen[volledige_naam] = 0
        path = os.path.join(spelers_dir, f'{voornaam}.{naam}.csv')
        self.bestanden[volledige_naam] = self._lees_csv(path)
        self.betaald[volledige_naam] = betaald

    def __str__(self):
        return self._genereer_klassementstekst()

    def _genereer_klassementstekst(self):
        rankschikking = sorted(self.personen.items(), key=lambda kv: -kv[1])
        regels = []
        regels.append(datetime.now().strftime("Laatst geüpdatet op: %d-%m-%Y %H:%M:%S"))

        huidige_rank, vorige_score = 1, None
        for index, (naam, score) in enumerate(rankschikking):
            if score != vorige_score:
                huidige_rank = index + 1
            suffix = '' if self.betaald[naam] else '*'
            regels.append(f'{huidige_rank}. {naam} {score}{suffix} ')
            vorige_score = score

        return '\n'.join(regels) + '\n'

    def set_punten(self):
        for speler in self.personen:
            bestand = self.bestanden[speler]

            for i in range(1, 7):
                for j in range(1, 7):
                    self.personen[speler] += self._punten(speler, i, j)

            self.personen[speler] += self._punten(speler, 9, 1)  # Bekerfinale

            for i in range(11, 18):  # Extra vragen
                if bestand[i][1].replace(" ", "") in [x.replace(" ", "") for x in self.juist[i][1].split(', ')] and bestand[i][1] != '':
                    self.personen[speler] += 10

            if bestand[18][1].replace(" ", "") == self.juist[18][1].replace(" ", "") and bestand[18][1] != '':
                self.personen[speler] += 20

        return self

    def _punten(self, speler, i, j):
        score = 0
        correct = self.juist[i][j].replace(" ", "")
        to_check = self.bestanden[speler][i][j].replace(" ", "")

        if len(to_check) == 3 and to_check[0].isdigit() and to_check[2].isdigit() and to_check[1] == '-':
            if len(correct) == 3 and correct[0].isdigit() and correct[2].isdigit() and correct[1] == '-':
                HC, AC = int(correct[0]), int(correct[2])
                H, A = int(to_check[0]), int(to_check[2])
                WC, GC, VC = HC > AC, HC == AC, HC < AC
                W, G, V = H > A, H == A, H < A

                if WC == W and GC == G and VC == V:
                    score += 5
                    if H - A == HC - AC:
                        score += 2
                        if HC == H and AC == A:
                            score += 3 + HC + AC
        elif i != j:
            print(f"WARN: something is wrong for player {speler} on row {i} column {j}")

        return score

    def write_to_textfile(self):
        with open('klassement.txt', 'w') as f:
            tekst = self._genereer_klassementstekst()
            f.write(tekst)

In [2]:
a = Pronostiek()
print(a.set_punten())
a.write_to_textfile()

Laatst geüpdatet op: 30-03-2025 18:07:25
1. Bart Winters 17 
1. Cedric Helewaut 17 
1. Hanne Hellemans 17 
1. Liina Vranckx 17 
1. Wouter Peeters 17 
6. Bram Desimpelaere 12 
6. Jarne Van Linden 12 
6. Jerry Teerlinck 12 
6. Lennert Lammens 12 
6. Nelly Van Acker 12 
6. Tom De Wispelaere 12 
6. Vicky D'haen 12 
6. Yvan Lammes 12 
14. Ann De Raeymaecker 10 
14. Bram Snoeys 10 
14. Dirk Vermeire 10 
14. Eddy Boone 10 
14. Elias Rito 10 
14. Evelien De Coninck 10 
14. Femke Desimpelaere 10 
14. Filip Marichal 10 
14. Gina De Maertelaere 10 
14. Jasper De Bruyne 10 
14. Jasper Maes 10 
14. Jef D'haen 10 
14. Lise Vandewalle 10 
14. Luc Claes 10 
14. Maarten Janssen 10 
14. Matts Van Nieuwenhove 10 
14. Merel Winters 10 
14. Pieter Meeuwissen 10 
14. Sam Gijsbrechts 10 
14. Tom De Coninck 10 
14. Wannes Debaene 10 
14. Wout Debussche 10 
14. Wout Pauwels 10 
37. Amaury Hellebuyck 7 
37. Arnaud De Groote 7 
37. Bart Debussche 7 
40. Cisse Debaene 5 
40. Eddy Wilms 5 
40. Gerben Van den Heuve